In [1]:
import numpy as np
import torch as pt
from torch import nn,autograd
from torch.autograd import Variable

In [3]:
X = np.array([[[0,1,0,0,1],
              [0,1,0,0,1],
              [1,0,0,0,1]],
             [[1,0,0,0,1],
              [1,0,0,0,1],
              [1,0,0,0,1]]])
X = X.reshape(2,5,3)
print(X.shape)
X

(2, 5, 3)


array([[[0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 0, 1]],

       [[1, 0, 0],
        [0, 1, 1],
        [0, 0, 0],
        [1, 1, 0],
        [0, 0, 1]]])

In [4]:
Xt = pt.from_numpy(X).type(pt.FloatTensor)

In [5]:
m = nn.Conv1d(5, 2, 1,padding=0)
input_ = Variable(Xt)

In [6]:
Xt


(0 ,.,.) = 
  0  1  0
  0  1  0
  1  0  0
  1  1  0
  0  0  1

(1 ,.,.) = 
  1  0  0
  0  1  1
  0  0  0
  1  1  0
  0  0  1
[torch.FloatTensor of size 2x5x3]

In [9]:
output = m(Variable(Xt))
output

Variable containing:
(0 ,.,.) = 
  1.3741  0.3741  0.3741
 -0.2942 -0.2942 -0.2942

(1 ,.,.) = 
  0.3741  0.3741  0.3741
 -0.2942 -0.2942 -0.2942
[torch.FloatTensor of size 2x2x3]

In [8]:
m.weight.data = pt.from_numpy(
    np.array([[[0],[0],[1],[0],[0]],[[0],[0],[0],[0],[0]]])
).type(pt.FloatTensor)

In [8]:
m.weight

Parameter containing:
(0 ,.,.) = 
  0
  0
  1
  0
  0

(1 ,.,.) = 
  0
  0
  0
  0
  0
[torch.FloatTensor of size 2x5x1]

### Road to actual Math Econ network

input data

In [2]:
def RuleGenerator(
    days=[0,0,0,0,0],
    products=[[0],[0],[0],[0],[0]],
    hours_of_preference=[9,16,9,16,9]
                 ):
    A = np.zeros((5,20))
    b = np.zeros((5,7))
    for i,d in enumerate(days,0):
        b[i][d] = hours_of_preference[i] + np.random.randint(0,4)
    for i,pl in enumerate(products,0):
        for p in pl:
            A[i,p] = 1
    return ([A[0:4][np.newaxis,:,:],b[0:4][np.newaxis,:,:]],
            [A[4].reshape(1,-1)[np.newaxis,:,:],b[4].reshape(1,-1)[np.newaxis,:,:]])

In [3]:
res = RuleGenerator(
    [1,2,3,1,2],
    [[1,2,3],[1,2,3,7],[1,2,3],[1,2,3,19],[1,2,3]]
)

In [159]:
class InstaNet(nn.Module):
    def __init__(self,batch_size,n_feat=20):
        super(InstaNet,self).__init__()
        self.batch_size = batch_size
        self.hidden = (Variable(pt.zeros(3,self.batch_size,15)),
                       Variable(pt.zeros(3,self.batch_size,15)))
        self.conv1 = nn.Conv1d(n_feat,15,1)
        self.conv2 = nn.Conv1d(15,10,1)
        self.lstm = nn.LSTM(10+7,15,3,batch_first=True)
        self.fcn0 = nn.Linear(4*15,300)
        self.fcn1 = nn.Linear(300,20)
        self.ReLU = nn.ReLU().forward
        
    def forward(self,x):
        out = self.conv1(x[0])
        out = self.ReLU(out)
        out = self.conv2(out)
        out = self.ReLU(out)
        out = pt.cat([out,x[1]],dim=1)
        out,self.hidden = self.lstm(out.transpose(2,1),self.hidden)
        out = self.fcn0(
            out.resize(self.batch_size,15*4)
        )
        out = self.ReLU(out)
        out = self.fcn1(out)
        out = nn.Sigmoid()(out)
        return out
    
    def init_hidden(self):
        self.hidden = (Variable(pt.zeros(3,self.batch_size,15)),
                       Variable(pt.zeros(3,self.batch_size,15)))

## dummy dataset creation

In [9]:
X1 = res[0][0]

In [10]:
X2 = res[0][1]

In [11]:
X1_t,X2_t = res[1][0],res[1][1]

In [13]:
np.r_[res[0][0],res[0][0]].shape

(2, 4, 20)

In [16]:
X1s = []
X2s = []
X1ts = []
X2ts = []

In [17]:
for i in range(200):
    res = RuleGenerator(
        [1,2,3,1,2],
        [[1,2,3],[1,2,3,7],[1,2,3],[1,2,3,19],[1,2,3]]
    )
    X1s.append(res[0][0])
    X2s.append(res[0][1])
    X1ts.append(res[1][0])
    X2ts.append(res[1][1])

In [18]:
for i in range(300):
    res = RuleGenerator(
        [2,5,5,2,5],
        [[11,17,19],[11,17,19,8],[11,17,19,8,3],[11,17,18,19],[11,17,18,19]]
    )
    X1s.append(res[0][0])
    X2s.append(res[0][1])
    X1ts.append(res[1][0])
    X2ts.append(res[1][1])

In [19]:
for i in range(200):
    res = RuleGenerator(
        [1,2,5,1,2],
        [[4,8,15],[4,8,15,17],[4,8,15,17,3],[4,8,15,17],[4,1,8,15]]
    )
    X1s.append(res[0][0])
    X2s.append(res[0][1])
    X1ts.append(res[1][0])
    X2ts.append(res[1][1])

In [21]:
X1 = pt.from_numpy(np.r_[X1s].reshape(-1,4,20)).type(pt.FloatTensor)
X2 = pt.from_numpy(np.r_[X2s].reshape(-1,4,7)).type(pt.FloatTensor)
X1t = pt.from_numpy(np.r_[X1ts].reshape(-1,1,20)).type(pt.FloatTensor)
X2t = pt.from_numpy(np.r_[X2ts].reshape(-1,1,7)).type(pt.FloatTensor)

In [168]:
criterion = pt.nn.NLLLoss()
optimizer = pt.optim.Adam(inst.parameters(),lr=0.01)
inst = InstaNet(100)

In [169]:
def random_sampler(dataset_size,batch_size=100):
    i = np.random.randint(0,dataset_size-batch_size)
    return (i,i+batch_size)

In [170]:
for epoch in range(10):
    optimizer.zero_grad()
    inst.init_hidden()
    i,j = random_sampler(700,100)
    pred = inst([Variable(X1[i:j].transpose(2,1)),Variable((X2[i:j]/24).transpose(2,1))])
    loss = criterion(pred.view(2000),Variable(X1t[i:j]).view(2000).type(pt.LongTensor))
    loss.backward()
    optimizer.step()
    print(pred)

Variable containing:
 0.5113  0.5055  0.5090  ...   0.4772  0.5098  0.5006
 0.5113  0.5055  0.5090  ...   0.4772  0.5098  0.5006
 0.5113  0.5055  0.5090  ...   0.4772  0.5098  0.5006
          ...             ⋱             ...          
 0.5089  0.5046  0.5078  ...   0.4790  0.5089  0.4996
 0.5089  0.5046  0.5078  ...   0.4790  0.5089  0.4996
 0.5089  0.5046  0.5078  ...   0.4790  0.5089  0.4996
[torch.FloatTensor of size 100x20]

Variable containing:
 0.5113  0.5055  0.5090  ...   0.4772  0.5098  0.5006
 0.5113  0.5055  0.5090  ...   0.4772  0.5098  0.5006
 0.5113  0.5055  0.5090  ...   0.4772  0.5098  0.5006
          ...             ⋱             ...          
 0.5089  0.5046  0.5078  ...   0.4789  0.5089  0.4997
 0.5089  0.5046  0.5078  ...   0.4789  0.5088  0.4997
 0.5089  0.5046  0.5078  ...   0.4789  0.5088  0.4997
[torch.FloatTensor of size 100x20]

Variable containing:
 0.5113  0.5055  0.5090  ...   0.4772  0.5098  0.5006
 0.5113  0.5055  0.5090  ...   0.4772  0.5098  0.5006
 